In [3]:
# from dataraw_sampling_SETShatter import *
from collections import defaultdict, Counter
import itertools
import copy
import random
import json
import operator as op
from functools import reduce
from tqdm import tqdm
from itertools import chain, combinations
import numpy as np
import time

In [ ]:
## Code -- can take in multiple tokens key now
# specify vocab_by_property

# Simple SET


# Construct Lookup
# Construct Resolve by Property

# 
# Sample cards on the fly
# 



# First do SET, and then SET with * in this notebook

In [4]:
# need decode_key_to_vocab_token


def construct_cardpair_answer_lookup(num_attributes, num_attr_vals, debug=False):
    '''Simple SET'''
    num_cards = num_attr_vals**num_attributes
    lookup = {}
    for card1, card2 in itertools.combinations(list(range(num_cards)), 2):
        card1_properties = decode_key_idx(num_attributes, num_attr_vals, card1)
        card2_properties = decode_key_idx(num_attributes, num_attr_vals, card2)
        card3, card3_properties = eval_cardpair_by_idx(num_attributes, num_attr_vals, card1, card2, debug=debug)
        if debug: print (card1_properties, card2_properties, card3_properties, card1, card2, card3)
        lookup[(card1, card2)] = card3
    for card in range(num_cards):
        lookup[(card, card)] = card
    return lookup

def sample_one_training_datapoint(num_keys, cardpair_answer_lookup):
    '''Simple SET'''
    card1_idx = np.random.choice(num_keys)
    card2_idx = np.random.choice(num_keys)

    q_vocab_tokens = [card1_idx, card2_idx]
    # int
    k_vocab_tokens = cardpair_answer_lookup[(card1_idx, card2_idx)]

    # list, list
    return q_vocab_tokens, [k_vocab_tokens]


In [5]:
def sample_queries(num_attributes, num_attr_vals, N_train, N_val, N_test):
    '''Simple SET'''

    num_keys = num_attr_vals**num_attributes
    N = N_train + N_val + N_test
    
    tokens = []
    gt_idxs = []

    cardpair_answer_lookup = construct_cardpair_answer_lookup(num_attributes, num_attr_vals)
    base_vocab_size =  num_keys
    symbol_vocab_token_lookup = {
        '(': base_vocab_size,
        ')': base_vocab_size + 1,
        'NULL': base_vocab_size + 2,
        'SEP': base_vocab_size + 3,
        'SOS': base_vocab_size + 4,
        'EOS': base_vocab_size + 5,
        'PAD': base_vocab_size + 6,
        'PLH': base_vocab_size + 7,
        '&': base_vocab_size + 8,
        '|': base_vocab_size + 9,
    }
    
    max_len_q = 2
    for i in tqdm(range(N)):
        q_vocab_tokens, k_vocab_tokens = sample_one_training_datapoint(num_keys, cardpair_answer_lookup)
        gt_ks_idx = k_vocab_tokens

        tokens.append((q_vocab_tokens, k_vocab_tokens))
        gt_idxs.append(gt_ks_idx)
        
        # stats
        max_len_q = max(max_len_q, len(q_tokens))

        data = {
            'num_attributes':num_attributes,
            'num_attr_vals':num_attr_vals,
            'nest_depth_int': None,
            'key_support_size': num_keys,
            'multiple_OR_sets_bool': None,

            'query_length_multiplier': None,
            'max_len_q': max_len_q,
            'len_k': 1,
            
            #################################        
            'train_gt_idxs': gt_idxs[:N_train],
            'val_gt_idxs': gt_idxs[N_train:N_train+N_val],
            'test_gt_idxs': gt_idxs[N_train+N_val:],
            
            'train_tokens': tokens[:N_train],
            'val_tokens': tokens[N_train:N_train+N_val],
            'test_tokens': tokens[N_train+N_val:],
            
            #################################

            'vocab_size': base_vocab_size + len(symbol_vocab_token_lookup),
            'symbol_vocab_token_lookup': symbol_vocab_token_lookup,
            'vocab_by_property': False,

            #################################
        }

        stats = {}

        return data, stats

In [6]:
data, stats = sample_queries(4, 3, N_train=100, N_val=100, N_test=100)

NameError: name 'decode_key_idx' is not defined

In [4]:
# Resolve one cardpair

def resolve_proppair(prop1, prop2, all_vals_tuple, all_vals_set):
    if prop1 == len(all_vals_tuple) or prop2 == len(all_vals_tuple): # '*' symbol
        # *
        return all_vals_tuple
    elif prop1 == prop2:
        return (prop1,)
    else:
        return tuple(all_vals_set - set([prop1, prop2]))

def resolve_cardpair(card1_prop, card2_prop, all_vals_list, all_vals_set):
    '''
    card1_prop/card2_prop: [1, 0 ,*], [2,0,1]
    return answer cards
    '''
    ind_ans = [resolve_proppair(prop1, prop2, all_vals_list, all_vals_set) for prop1, prop2 in zip(card1_prop, card2_prop)]
    return set(itertools.product(*ind_ans))


def draw_cardpair(num_attributes, num_attr_vals):
    all_vals_tuple = tuple(range(num_attr_vals))
    all_vals_set = set(all_vals_tuple)
    
    card1_prop = np.random.choice(a=num_attr_vals, size=num_attributes, replace=True)
    card2_prop = np.random.choice(a=num_attr_vals+1, size=num_attributes, replace=True)
    subset = resolve_cardpair(card1_prop, card2_prop, all_vals_tuple, all_vals_set)
    # list of tuples, each one card
    return subset

In [22]:
set_fns = {
    '|': lambda x,y: x | y,
    '&': lambda x,y: x & y,
    '-': lambda x,y: x - y,
    '!': lambda x,y: x.symmetric_difference(y)
}

def draw_two_cardpairs(num_attributes, num_attr_vals):
    all_subsets = []
    subset1 = draw_cardpair(num_attributes, num_attr_vals)
    subset2 = draw_cardpair(num_attributes, num_attr_vals)
    op = np.random.choice(a=('|', '&', '-', '!'))
#     print(subset1, op, subset2)
    ans_subset = set_fns[op](subset1, subset2)
    return ans_subset, len(ans_subset)

In [33]:
all_ans_subsets = set()
all_ans_subsets_count = defaultdict(int)
ans_subset_sizes = []

for _ in range(10000):
    # set, int
    ans_subset, ans_subset_size = draw_two_cardpairs(num_attributes=4, num_attr_vals=3)
    ans_subset = tuple(sorted(ans_subset))
    all_ans_subsets.add(ans_subset)
    all_ans_subsets_count[ans_subset] += 1
    
print('Number of card subsets generated =', len(all_ans_subsets))

Number of card subsets generated = 4918


In [2]:
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom